# Manually Logging Nested Traces with TraceSpanDict and StreamTable


This documentation provides a guide on how to use `TraceSpanDict` and `StreamTable` for manual logging of nested traces in your LLM applications.

# Overview

In machine learning, particularly when working with large language models (LLMs), it's crucial to track the flow of information and performance across different components. W&B’s Traces tool is instrumental in visualizing and understanding the interactions within LLM chains, encompassing aspects like input/output data, execution flow, model architecture, and intermediate results.

## Key Concepts

`Trace`: A trace is a collection of spans that records a structured operation, such as data retrieval or processing within an LLM chain. It's a tool to visualize the flow and integration of components in LLM pipelining or chaining.

`Span`: In the context of Traces, a span represents an individual operation or processing segment in the LLM chain, like a document lookup or a model inference step. Spans are uniquely identified (span_id) and can be linked hierarchically.

## Components

`TraceSpanDict`: This data structure encapsulates a span's details — capturing aspects like the operation name, unique identifiers, timestamps, and parent-child relationships among spans.

`StreamTable`: Used for logging and monitoring spans, StreamTable helps in reviewing the overall inputs and outputs, and in tracing the execution flow within an LLM chain.


- `name`: The name of the span (e.g., 'HTTPRequest').
- `span_id`: A unique identifier for the span.
- `trace_id`: The identifier of the trace this span belongs to.
- `parent_id`: The ID of this span's parent span (if any).
- `start_time_s`: The start time of the span (in seconds).
- `end_time_s`: The end time of the span (in seconds).
Other span attributes, such as status codes, inputs, outputs, etc.


## Quickstart Guide
1. Installation

Ensure you have the required packages installed:

In [ ]:
!pip install wandb weave

2. Log in to W&B

In [ ]:
import wandb
wandb.login()

3. Logging a Trace with Nested Spans

In [ ]:
WB_PROJECT = 'your_project_name'
WB_STREAM = 'your_stream_name'

from uuid import uuid4
import datetime
from weave.monitoring import StreamTable
from weave.stream_data_interfaces import TraceSpanDict


## Log Nested Spans

Here’s how to log a simple nested trace:

Each operation within the trace can be logged as a nested span, with parent_id set to the ID of the immediate parent span. You can tailor the `attributes`, `inputs`, `outputs`, and `summary` fields in your trace logs to match the inputs and outputs of your system. Here's an example setup:

In [ ]:
trace_id = str(uuid4()) # top level grouping
root_span_id = str(uuid4()) # used as parent_id for nested spans
prev_end_time = None
start_time = datetime.datetime.now().timestamp()
for i in range(10):
    nested_span_id = str(uuid4())
    nested_start_time = prev_end_time if prev_end_time is not None else start_time
    end_time = datetime.datetime.now().timestamp()
    nested_span = TraceSpanDict(
        name='NestedOperation',
        span_id=nested_span_id,
        trace_id=trace_id,
        parent_id=root_span_id, # set to None if you want to automatically create a root span in the UI
        start_time_s=nested_start_time,
        end_time_s=end_time,
        status_code='SUCCESS',
        attributes={},
        inputs=dict(input='hello nested',params={"temp": 0.7}),
        outputs=dict(output="hello to you 2"),
        summary={},
        exception=None,
    )
    prev_end_time = end_time
    st.log(nested_span)

If you don't define a `parent_id`, all logged spans linked by the same `trace_id` will be part of the same trace and the UI will show a root span. Alternatively, as we did above, you can choose to group your spans yourself by logging the root span manually and using it's `span_id` as the `parent_id` of nested spans, this gives you greater control of the top level information.

# Log the root span

In [ ]:
root_span = TraceSpanDict(
    name='Exchange',
    span_id=root_span_id,
    trace_id=trace_id,
    parent_id=None, # logged top level span has parend_id=None
    start_time_s=start_time,
    end_time_s=datetime.datetime.now().timestamp(),
    status_code='SUCCESS',
    attributes=dict(
        username='Geoffrey',
        prompt='you are a chatbot',
        operation_type="Chat" 
    ),
    inputs=dict(input='hello',params={"temp": 0.7}),
    outputs=dict(output="hello to you"),
    summary={},
    exception=None,
)
st.log(root_span)

## Conclusion

This guide helps you manually log nested traces in your applications using `TraceSpanDict` and `StreamTable`. Proper implementation of tracing provides deep insights into the performance and behavior of your applications, aiding in debugging and performance optimization when using LLMs.